In [1]:
import pandas as pd
from datetime import timedelta
import os

try:
    import fastf1 as ff1
    from fastf1 import plotting
except ImportError:
    raise ImportError(
        "Il pacchetto fastf1 non è installato. Installalo con: pip install fastf1"
    )

# Configurazione iniziale
YEAR = 2024
EVENT_NAME = "Italian Grand Prix"
DRIVER_NUMBER = "16"  # Esempio: Charles Leclerc
CACHE_DIR = "fastf1_cache"  # Cartella per la cache dei dati

def setup_environment():
    """Configura l'ambiente e la cache"""
    if not os.path.exists(CACHE_DIR):
        os.makedirs(CACHE_DIR)
    ff1.Cache.enable_cache(CACHE_DIR)
    plotting.setup_mpl()

def load_session_data():
    """Carica i dati della sessione con gestione errori avanzata"""
    try:
        session = ff1.get_session(YEAR, EVENT_NAME, 'R')
        session.load(telemetry=False, weather=False, messages=False)
        return session
    except Exception as e:
        print(f"Errore nel caricamento dati: {str(e)}")
        print("Possibili soluzioni:")
        print("1. Verifica la connessione a internet")
        print("2. Controlla che l'evento esista per l'anno specificato")
        print("3. Prova a cancellare la cache e riprovare")
        return None

def process_sector_data(session):
    """Elabora i dati dei settori"""
    if not session:
        return None

    laps = session.laps
    valid_laps = laps[
        (laps['IsAccurate']) & 
        (laps['LapTime'].notna()) & 
        (laps['PitOutTime'].isna()) & 
        (laps['PitInTime'].isna())
    ].copy()

    sectors_list = []
    for sector_num in range(1, 4):
        sector_col = f'Sector{sector_num}Time'
        session_time_col = f'Sector{sector_num}SessionTime'
        
        sector_laps = valid_laps[valid_laps[sector_col].notna()].copy()
        sector_laps['Sector'] = sector_num
        sector_laps['SectorTime'] = sector_laps[sector_col]
        sector_laps['SectorSessionTime'] = sector_laps[session_time_col]
        
        sectors_list.append(
            sector_laps[['DriverNumber', 'LapNumber', 'Sector', 
                        'SectorTime', 'SectorSessionTime']]
        )

    return pd.concat(sectors_list)

def analyze_driver_performance(sector_data, driver_number):
    """Analizza le prestazioni del pilota"""
    if sector_data is None:
        return None

    # Calcola la posizione per ogni settore
    sector_data['Position'] = sector_data.groupby(
        ['LapNumber', 'Sector']
    )['SectorTime'].rank()

    driver_data = sector_data[
        sector_data['DriverNumber'] == driver_number
    ].copy()

    results = []
    for _, row in driver_data.iterrows():
        lap = row['LapNumber']
        sector = row['Sector']
        pos = row['Position']

        # Trova piloti vicini
        sector_lap_data = sector_data[
            (sector_data['LapNumber'] == lap) & 
            (sector_data['Sector'] == sector)
        ]

        ahead = sector_lap_data[
            sector_lap_data['Position'] == pos - 1
        ]
        behind = sector_lap_data[
            sector_lap_data['Position'] == pos + 1
        ]

        results.append({
            'LapNumber': lap,
            'Sector': sector,
            'Position': pos,
            'SectorTime': row['SectorTime'],
            'SectorSessionTime': row['SectorSessionTime'],
            'DriverAhead': ahead['DriverNumber'].values[0] if not ahead.empty else None,
            'TimeAhead': ahead['SectorTime'].values[0] if not ahead.empty else None,
            'DriverBehind': behind['DriverNumber'].values[0] if not behind.empty else None,
            'TimeBehind': behind['SectorTime'].values[0] if not behind.empty else None
        })

    return pd.DataFrame(results)

def calculate_metrics(results_df):
    """Calcola metriche aggiuntive"""
    if results_df is None:
        return None

    results = results_df.copy()
    
    # Calcola delta in secondi
    results['DeltaToAhead'] = results.apply(
        lambda x: (x['SectorTime'] - x['TimeAhead']).total_seconds() 
        if pd.notna(x['TimeAhead']) else None, axis=1
    )
    
    results['DeltaToBehind'] = results.apply(
        lambda x: (x['SectorTime'] - x['TimeBehind']).total_seconds() 
        if pd.notna(x['TimeBehind']) else None, axis=1
    )
    
    # Crea timestamp di mezzo secondo
    results['HalfSecondTimestamp'] = results['SectorSessionTime'].dt.floor('500ms')
    
    return results

def save_and_show_results(results):
    """Salva e mostra i risultati"""
    if results is None:
        return

    output_file = f"sector_analysis_{EVENT_NAME}_{YEAR}_driver_{DRIVER_NUMBER}.csv"
    results.to_csv(output_file, index=False)
    print(f"Dati salvati in {output_file}")
    
    print("\nAnteprima dei risultati:")
    print(results[[
        'LapNumber', 'Sector', 'Position', 'SectorTime',
        'DriverAhead', 'DeltaToAhead', 'DriverBehind', 'DeltaToBehind'
    ]].head())

def main():
    """Funzione principale"""
    setup_environment()
    
    print(f"Caricamento dati per {EVENT_NAME} {YEAR}...")
    session = load_session_data()
    
    if not session:
        return

    print("Elaborazione dati dei settori...")
    sector_data = process_sector_data(session)
    
    print(f"Analisi prestazioni pilota {DRIVER_NUMBER}...")
    driver_results = analyze_driver_performance(sector_data, DRIVER_NUMBER)
    
    final_results = calculate_metrics(driver_results)
    save_and_show_results(final_results)

if __name__ == "__main__":
    main()

c:\Users\domen\OneDrive\Desktop\Develhope\Projects\Github\F1 Team Project\FASTF1\.venv\Lib\site-packages\fastf1\plotting\_plotting.py:90: FutureWarning: FastF1 will no longer silently modify the default Matplotlib colors in the future.
To remove this warning, explicitly set `color_scheme=None` or `color_scheme='fastf1'` when calling `.setup_mpl()`.
  warnings.warn(
c:\Users\domen\OneDrive\Desktop\Develhope\Projects\Github\F1 Team Project\FASTF1\.venv\Lib\site-packages\fastf1\plotting\_plotting.py:98: FutureWarning: FastF1 will stop modifying the default Matplotlib settings in the future.
To opt-in to the new behaviour and remove this warning, explicitly set `misc_mpl_mods=False` when calling `.setup_mpl()`.
  warnings.warn(


Caricamento dati per Italian Grand Prix 2024...


core           INFO 	Loading data for Italian Grand Prix - Race [v3.5.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

Elaborazione dati dei settori...
Analisi prestazioni pilota 16...
Dati salvati in sector_analysis_Italian Grand Prix_2024_driver_16.csv

Anteprima dei risultati:
   LapNumber  Sector  Position             SectorTime DriverAhead  \
0        2.0       1       4.0 0 days 00:00:27.707000          44   
1        3.0       1      11.0 0 days 00:00:27.679000          18   
2        4.0       1       8.0 0 days 00:00:27.653000          44   
3        5.0       1       5.0 0 days 00:00:27.630000          55   
4        6.0       1       5.0 0 days 00:00:27.596000          44   

   DeltaToAhead DriverBehind  DeltaToBehind  
0         0.006           81         -0.044  
1         0.008            1         -0.052  
2         0.003           27         -0.010  
3         0.045           18         -0.006  
4         0.031           43         -0.037  
